# MongoDB and Python

MongoDB is a 'NoSQL database' with support for high-performance document-oriented storage and queries, sharding, and replication.

Terminology:

- A **document** is a single JSON-like object stored in MongoDB
- A **collection** is a respository of documents which may have one or more indexes on them
- A **database** is a group of collections and indexes 


To get started, we'll install the `pymongo` driver and the `dnspython` modules to allow us to use the "mongodb+srv://" URLs to connect to MongoDB:

In [ ]:
!pip install -U pymongo dnspython

## Connecting and accessing databases and collections

In [ ]:
import pymongo
username = 'class'
password = 'classword'
host = 'training.i7auh.mongodb.net'
dbname = 'training'
cli = pymongo.MongoClient(
    f'mongodb+srv://{username}:{password}@{host}'#'/{dbname}'
)
#cli = pymongo.MongoClient('mongodb://localhost:27017/class')

In [ ]:
cli

In [ ]:
db = cli['training']
db

In [ ]:
cli.training

In [ ]:
for cname in db.list_collection_names():
    print(cname)
    db[cname].drop()

## Inserting data

https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html

In [ ]:
res = db.roster.insert_one({
    'name': 'Rick Copeland',
    'email': 'edward.fine@afinepoint.net',
    'role': 'Instructor',
})
res

In [ ]:
res.inserted_id

In [ ]:
res.inserted_id.generation_time

## Querying data

[Additional documentation: query operators](https://docs.mongodb.com/manual/reference/operator/query/)

[Additional documentation: find](http://api.mongodb.com/python/current/api/pymongo/collection.html#pymongo.collection.Collection.find)

[Additional documentation: find_one](http://api.mongodb.com/python/current/api/pymongo/collection.html#pymongo.collection.Collection.find_one)

In [ ]:
cursor = db.roster.find({'name': 'Rick Copeland'})  # {'name': {'$eq': 'Rick Copeland'}}
cursor

In [ ]:
list(cursor)

In [ ]:
for doc in db.roster.find({'name': {'$eq': 'Rick Copeland'}}):
    print(doc)

In [ ]:
for item in db.roster.find({'name': {'$gte': 'Rick Copeland'}}):
    print(item)

In [ ]:
for item in db.roster.find({'roles.name': 'instructor'}):
    print(item)

In [ ]:
doc = db.roster.find_one({'_id': item['_id']})
doc

In [ ]:
import re
db.roster.find_one({'role': re.compile(b'^Ins')})  # SQL equivalent: LIKE 'Ins%'

In [ ]:
import re
db.roster.find_one({'role': {'$regex': '^Ins'}})  # SQL equivalent: LIKE 'Ins%'

In [ ]:
db.roster.index_information().keys()

In [ ]:
db.roster.create_index([
    ('role', 1),
])
db.roster.find({'role': re.compile(b'^Ins')}, {'_id': 0, 'role': 1}).explain()

## Updating data

[Additonal documentation: update operators](https://docs.mongodb.com/manual/reference/operator/update/)

[Additional documentation: replace](http://api.mongodb.com/python/current/api/pymongo/collection.html#pymongo.collection.Collection.replace_one)

[Additional documentation: update](http://api.mongodb.com/python/current/api/pymongo/collection.html#pymongo.collection.Collection.update_one)

In [ ]:
oid = doc['_id']

In [ ]:
doc['email'] = 'rick446@arborian.com'
db.roster.replace_one(
    {'_id': oid},
    doc
)

In [ ]:
doc = db.roster.find_one(dict(_id=oid))
doc

In [ ]:
db.roster.update_one(
    {'_id': oid},
    {'$set': {'email': 'edward.fine@afinepoint.net'}}
)

In [ ]:
doc = db.roster.find_one({'_id': oid})
doc

## Atomic find/modify

In [ ]:
coll = db.roster
doc = coll.find_one_and_update(
    {'name': 'Rick Copeland'},
    {'$inc': {'classes': 1}},
    return_document=pymongo.ReturnDocument.AFTER,
    upsert=True,
)
doc

In [ ]:
coll = db.roster
doc = coll.find_one_and_update(
    {'name': 'Ricardo Copeland'},
    {'$inc': {'classes': 1}},
    return_document=pymongo.ReturnDocument.AFTER,
    upsert=True,
)
doc

In [ ]:
list(db.roster.find())

Psuedocode for locking using MongoDB

```python

{
    waiters: [
        ...
    ]
}

lock = db.lock.find_one_and_update(
    {_id: 'mylock'},
    {'$push': {'waiters': my_unique_id}}
    return_document=pymongo.ReturnDocument.AFTER
)

if lock.waiters[0] == my_id:
    I got it!
```

## Delete

In [ ]:
import re

res = coll.delete_one({'name': re.compile(r'^Ri')})
res

In [ ]:
list(coll.find())

In [ ]:
second_res = coll.delete_many({'name': re.compile(r'^Ri')})

In [ ]:
res.deleted_count

In [ ]:
second_res.deleted_count

In [ ]:
list(coll.find())

Using pandas with PyMongo?

In [ ]:
import pandas as pd
stock = pd.read_csv('data/closing-prices.csv', parse_dates=[0])
stock.head()

In [ ]:
records = stock.to_dict(orient='records')

In [ ]:
records[0]

In [ ]:
res = db.stock.insert_many(records)
res

In [ ]:
len(res.inserted_ids)

In [ ]:
db.stock.find_one()

In [ ]:
from datetime import datetime
db.stock.find_one({'Unnamed: 0': datetime(2014,1,2)})

Open [PyMongo Lab](./pymongo-lab.ipynb)